In [17]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass

# for configuring connection 
from configobj import ConfigObj
import os

# local utils
import utils

%matplotlib inline

In [18]:
# Create a database connection using settings from config file
config='db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con_eicu = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con_eicu = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con_eicu = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
eicu_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: alistairewj


In [19]:
# Create a database connection using settings from config file
config='db/config-mimic.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con_mimic = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con_mimic = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con_mimic = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
mimic_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: mimic
Username: alistairewj


## Same as above - but 24 hour matrices

In [20]:
print('=== eICU ===')
query = eicu_schema + """
select co.*, tm.windowtime_hours
from tr_cohort co
left join tr_time_day1 tm
on co.patientunitstayid = tm.patientunitstayid
"""
co_eicu = pd.read_sql_query(query, con_eicu)
co_eicu.set_index('patientunitstayid', inplace=True)

co_eicu = utils.drop_patients(co_eicu)

query = eicu_schema + """
select co.patientunitstayid
  , aiva.acutephysiologyscore as apsiii
from tr_cohort co
left join (select patientunitstayid, acutephysiologyscore from APACHEPATIENTRESULT where apacheversion = 'IVa') aiva
on co.patientunitstayid = aiva.patientunitstayid
order by 1, 2
"""
df_eicu = pd.read_sql_query(query, con_eicu)
df_eicu.set_index('patientunitstayid', inplace=True)

# static data
query = eicu_schema + """select * from tr_static_data"""
df_eicu_static = pd.read_sql_query(query, con_eicu)
df_eicu_static.set_index('patientunitstayid', inplace=True)

# add in outcomes/static vars to the design matrix
vars_static = ['death', 'ventdays',
               'is_female', 'age',
               'race_black', 'race_hispanic', 'race_asian', 'race_other',
               'electivesurgery']
X_eicu = df_eicu.merge(df_eicu_static[vars_static], how='inner',
                       left_index=True, right_index=True)

# add in hospitalid/outcomes to start of dataframe
vars_outcome = ['hospitalid', 'hosp_los']
X_eicu = co_eicu[vars_outcome].merge(X_eicu, how='inner',
              left_index=True, right_index=True)

# rearrange certain columns to beginning of dataframe
# this is only for aesthetics - nice to have non-inputs at start of df
vars_outcome = ['hospitalid', 'death', 'hosp_los', 'ventdays']
X_eicu = X_eicu[ vars_outcome + [c for c in X_eicu.columns if c not in vars_outcome] ]
X_eicu.head()

# write to file
X_eicu.to_csv('X_eicu_apsiii.csv.gz', compression='gzip')
print('Exported {} rows to file.'.format(X_eicu.shape[0]))

=== eICU ===
   181 removed due to exclusion_non_adult
 16311 removed due to exclusion_bad_data
     0 removed due to exclusion_organ_donor
101010 removed due to exclusion_by_apache

107100 (53.32%) removed so far.
  extra  23626 removed due to window time.

 70133 (34.92%) - final cohort size.
Exported 70133 rows to file.


In [21]:
print('=== MIMIC ===')

query = mimic_schema + """
select co.*, tm.windowtime_hours
from tr_cohort co
left join tr_time_day1 tm
on co.icustay_id = tm.icustay_id"""
co_mimic = pd.read_sql_query(query, con_mimic)
co_mimic.set_index('icustay_id', inplace=True)
co_mimic = utils.drop_patients(co_mimic)

query = mimic_schema + """
select co.icustay_id
  , a.apsiii
from tr_cohort co
left join apsiii a
  on co.icustay_id = a.icustay_id
order by 1, 2
"""
df_mimic = pd.read_sql_query(query, con_mimic)
df_mimic.set_index('icustay_id', inplace=True)

# get static data
query = mimic_schema + """select * from tr_static_data"""
df_mimic_static = pd.read_sql_query(query, con_mimic)
df_mimic_static.set_index('icustay_id', inplace=True)

# add in outcomes/static vars to the design matrix
vars_static = ['death', 'ventdays',
               'is_female', 'age',
               'race_black', 'race_hispanic', 'race_asian', 'race_other',
               'electivesurgery']
X_mimic = df_mimic.merge(df_mimic_static[vars_static], how='inner',
                       left_index=True, right_index=True)

# add in hospitalid/outcomes to start of dataframe
vars_outcome = ['hospitalid', 'hosp_los']
X_mimic = co_mimic[vars_outcome].merge(X_mimic, how='inner',
              left_index=True, right_index=True)

# rearrange certain columns to beginning of dataframe
# this is only for aesthetics - nice to have non-inputs at start of df
vars_outcome = ['hospitalid', 'death', 'hosp_los', 'ventdays']
X_mimic = X_mimic[ vars_outcome + [c for c in X_mimic.columns if c not in vars_outcome] ]
X_mimic.head()


# write to file
X_mimic.to_csv('X_mimic_apsiii.csv.gz', compression='gzip')
print('Exported {} rows to file.'.format(X_mimic.shape[0]))

=== MIMIC ===
  8109 removed due to exclusion_non_adult
  1347 removed due to exclusion_bad_data
     4 removed due to exclusion_organ_donor
 18691 removed due to exclusion_by_apache

 23825 (38.72%) removed so far.
  extra   9767 removed due to window time.

 27940 (45.41%) - final cohort size.
Exported 27940 rows to file.


## Close DB connections

In [22]:
con_mimic.close()
con_eicu.close()